# Structural analysis with MDAnalysis

In [ ]:
import matplotlib.pyplot as plt
import re
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis import distances

### Read the pdb in MDAnalysis

In [ ]:
PDB_ = "BANAL-20-52.B99990001.pdb"
u = mda.Universe(PDB_)

### Select the atoms for distance calculation
##### We will select carbon alphas of our protein and of the antibody

In [ ]:
# get all CA atoms
CA_atoms = u.select_atoms('name CA')

# get the CA atoms of spike and from the antibody Fab domain. You must edit the chains that must be selected if necessary
spike_CA = CA_atoms.select_atoms('chainID A')
# get the CA atoms of the antibody Fab domain. This all the other chains wrt to the spike chain
antibody_CA = CA_atoms.select_atoms('not chainID A')

print("spike number of residues: {}".format(len(spike_CA)))
print("antibody number of residues: {}".format(len(antibody_CA)))

### Calculate the distance matrix between these two selections

In [ ]:
distance_matrix = distances.distance_array(spike_CA, antibody_CA)
print(distance_matrix.shape)

### Set a reasonable distance cutoff for contact:

In [ ]:
distance_cutoff = 8.0

### Identify the contacts

In [ ]:
ids = np.where(distance_matrix < distance_cutoff)
# contact residues
contacts = sorted(list(set(ids[0])))

print("{} residues were found at the interation surface with the antibody".format(len(contacts)))

### Save interfacial residues to file
##### We can run this on multiple PDBs and plot statistics of frequency

In [ ]:
OUTFILE_ = re.sub(".pdb$",".inter", PDB_)
out = open(OUTFILE_, "w")
for id in contacts:
    at = spike_CA[id]
    out.write("%s %d %s\n" % (at.resname, at.resid, at.segid))
out.close()

### Create a PDB file with interaction information in Bfactor column

In [ ]:
# set all the bfactors to zero
atoms = u.atoms
for at in atoms:
    at.tempfactor = 0.0
# set interacting residues on viral protein to a large number (100.0)
for id in contacts:
    r = spike_CA[id].residue
    for at in r.atoms:
        at.tempfactor = 100.0
# save PDB file
OUTFILE_ = re.sub(".pdb$",".inter.pdb", PDB_)
atoms.write(OUTFILE_)